Carga el `libro1.txt` que contiene el texto del primer libro de Harry Potter, a partir de la línea 38, para realizar los siguientes ejercicios.

In [14]:
with open("libro1.txt", "r", encoding="utf-8") as archivo:
    lineas = archivo.readlines()[37:]  # Desde la línea 38
    texto = "".join(lineas)



Carga el modelo `es_core_news_md` de `spaCy`. Crea un `doc` con el libro del texto.

In [15]:
import spacy
nlp = spacy.load("es_core_news_md")
doc = nlp(texto)

## Ejercicio 1

Extrae todas las entidades nombradas de tipo `PER` (persona) y muestra la lista de personajes encontrados, seguida de los personajes más mencionados junto con su frecuencia.

In [16]:
#extrae todas las entidades cuya label es PER y almacenalas en un contador
from collections import Counter
from spacy.tokens import Span   
#crea el objeto contador
contador = Counter()
#extrae las entidades PER
for ent in doc.ents:
    if ent.label_ == "PER":
        contador[ent.text] += 1
#imprime la lista de todos los personajes
for personaje in contador:
    print(personaje)
#imprime la lista de personajes más mencionados con su frecuencia
print("Personajes en el libro:")
for personaje, cantidad in contador.most_common():
    print(f"({personaje}, {cantidad}) ")


Dursley
Dudley
Sabían
Tunante
Podría
Harvey
Jim McGuffin
Ted
Telespectadores
Lechuzas
Aquello
Dursley. Lechuzas
Howard
Petunia
Apenas
Albus Dumbledore
Parecía
Dumbledore
McGonagall
Estrellas
Dedalus Diggle
Nunca
Voldemort
Pomfrey
Sabe
Lily
James Potter
James
Dicen
McGonagall. Después
Sí
Son
Hagrid
Cabello
Sirius Black
Entonces
Sirius
Aquél
Dudley Dursley
Había
quiso
date prisa
Duddy
Tío Vernon
tío Vernon
Vernon
Tía Petunia
tía Marge
Bravo
tía Petunia
Vernon dijo. La señora Figg
Figg
Tibbles
Snowy
Señor Paws
Tufty
Marge sugirió tío Vernon
Yvonne
Dudley
Tía Petunia
Piers Polkiss
Piers
Parecían
Podía
Lenta
Miró
Dudley Después
Dennis
Gordon
Stonewall
Debían
Creyó
Marge
Señor H. Potter
Alacena Debajo de la Escalera
Privet Drive
Entregó
Golpeó
Ganó Dudley
Harry enfadado.
Échalo
Señor H. Potter
Allí
Veinticuatro
Podían
Ya
Señor H. Potter
Habitación
Claro
Esperaba
Feliz Cumpleaños
Rubeus Hagrid
Guardián de las Llaves
Harry
Hagrid
Sabía
Vernon Dursley
Gran Hechicero
Jefe de Magos
Miner

## Ejercicio 2

Detecta oraciones que parecen ser diálogos, es decir, que comienzan por guión

In [17]:
#separa el texto en oraciones
oraciones = list(doc.sents)
#detecta las oraciones que empiezan por '-'
oraciones_guiones = [oracion for oracion in oraciones if oracion.text.startswith(" ")]
#imprime las oraciones que empiezan por '-'
for oracion in oraciones_guiones:
    print(oracion.text)
#sale vacio porque los guiones están mal codificados

## Ejercicio 3

Detecta menciones de las casas del colegio hogwarts 'Gryffindor' y 'Slytherin' usando `Matcher`.

In [18]:
#usando matcher crea una lista de patrones para detectar las oraciones que contienen 'Gryffindor' y 'Slytherin'
from spacy.matcher import Matcher
matcher = Matcher(nlp.vocab)
#patrón para Gryffindor
patron1 = [{"LOWER": "gryffindor"}]
#patrón para Slytherin
patron2 = [{"LOWER": "slytherin"}]
#añade los patrones al matcher
matcher.add("Gryffindor", [patron1])
matcher.add("Slytherin", [patron2])
#aplica el matcher al documento
matches = matcher(doc)
#imprime las oraciones que contienen 'Gryffindor' y 'Slytherin'
for match_id, start, end in matches:
    span = doc[start:end]
    #imprime la oración que contiene el patrón
    print(span.sent.text)

Bueno, nadie lo sabrá realmente hasta que lleguemos allí, pero yo sé que seré de Slytherin, porque toda mi familia fue de allí.
¿Y qué son Slytherin y Hufflepuff?
Casas del colegio.
Es mejor Hufflepuff que Slytherin dijo Hagrid con tono lúgubre. Las brujas y los magos que se volvieron malos habían estado todos en Slytherin.
Es mejor Hufflepuff que Slytherin dijo Hagrid con tono lúgubre. Las brujas y los magos que se volvieron malos habían estado todos en Slytherin.
Estuve preguntando por ahí y espero estar en Gryffindor, parece la mejor de todas.
No creo que Ravenclaw sea tan mala, pero imagina si me ponen en Slytherin.

»Las cuatro casas se llaman Gryffindor, Hufflepuff, Ravenclaw y Slytherin.
»Las cuatro casas se llaman Gryffindor, Hufflepuff, Ravenclaw y Slytherin.
Puedes pertenecer a Gryffindor, 
donde habitan los valientes.

Su osadía, temple y caballerosidad
ponen aparte a los de Gryffindor.

O tal vez en Slytherin
harás tus verdaderos amigos.

Brocklehurst, Mandy tambié

## Ejercicio 4

Extrae adjetivos cercanos a entidades de tipo `LOC` para entender cómo se describen los lugares.

In [19]:

# Diccionario estándar
resultado = {}

for ent in doc.ents:
    if ent.label_ == "LOC":
        adyacentes = []

        start_idx = ent.start
        end_idx = ent.end

        # Buscar token anterior
        if start_idx > 0:
            token_antes = doc[start_idx - 1]
            if token_antes.pos_ == "ADJ":
                adyacentes.append(token_antes.text)

        # Buscar token posterior
        if end_idx < len(doc):
            token_despues = doc[end_idx]
            if token_despues.pos_ == "ADJ":
                adyacentes.append(token_despues.text)

        if adyacentes:
            clave = ent.text
            # Si ya existe esa clave con esos adjetivos, agregamos más sin repetir
            if clave in resultado:
                for adj in adyacentes:
                    if adj not in resultado[clave]:
                        resultado[clave].append(adj)
            else:
                resultado[clave] = adyacentes

# Mostrar resultados
for loc, adjs in resultado.items():
    print(f"{loc} -> {adjs}")

Mitad -> ['aterrorizados']
Gryffindor -> ['nueva']
Patrullaba -> ['sola']
Fuimos -> ['aplastados']
Pince -> ['pegada']
Estaba -> ['envuelto']
Piedra -> ['única']
la Piedra -> ['\x97¿Entonces']


## Ejercicio 5

Extrae verbos relacionados con las emociones que se listan a continuación y cuenta su frecuencia.

```
emociones = ["temer", "amar", "odiar", "llorar", "reír", "sufrir", "sentir"]
```

In [20]:
#creamos un objeto Counter
contador = Counter()
#extraemos los verbos relacionados con las emociones= ["temer", "amar", "odiar", "llorar", "reír", "sufrir", "sentir"] y los almacenamos en el counter
# Verbos de emociones a buscar
verbos_emocion = {"temer", "amar", "odiar", "llorar", "reír", "sufrir", "sentir"}

# Buscamos los verbos relacionados con emociones
for token in doc:
    if token.pos_ == "VERB":
        # Normalizamos el verbo a su forma base (lema)
        lema = token.lemma_.lower()
        if lema in verbos_emocion:
            contador[lema] += 1
print(contador)


Counter({'sentir': 75, 'reír': 18, 'llorar': 16, 'temer': 8, 'odiar': 5, 'sufrir': 4, 'amar': 2})


## Ejercicio 6

Extrae oraciones que contengan al menos dos entidades nombradas (`PER`, `LOC`, `ORG`).

In [21]:
'''#separar el texto en oraciones
oraciones = list(doc.sents)
#almacena las oraciones que contengan al menos dos entidades cuya label sea PER, LOC u ORG
oraciones_con_dos_entidades = []
for oracion in oraciones:
    entidades = [ent for ent in oracion.ents if ent.label_ in ["PER", "LOC", "ORG"]]
    if len(entidades) >= 2:
        oraciones_con_dos_entidades.append(oracion.text)
#imprime las oraciones que contienen al menos dos entidades
#oraciones_con_dos_entidades[0:5]'''

'#separar el texto en oraciones\noraciones = list(doc.sents)\n#almacena las oraciones que contengan al menos dos entidades cuya label sea PER, LOC u ORG\noraciones_con_dos_entidades = []\nfor oracion in oraciones:\n    entidades = [ent for ent in oracion.ents if ent.label_ in ["PER", "LOC", "ORG"]]\n    if len(entidades) >= 2:\n        oraciones_con_dos_entidades.append(oracion.text)\n#imprime las oraciones que contienen al menos dos entidades\n#oraciones_con_dos_entidades[0:5]'

## Ejercicio 7

Busca oraciones que contengan palabras explicativas, como 'como', 'parecía', 'tal que'.

In [22]:
#creamos un conjunto con las palabras explicativas como, parecía y tal que
palabras_explicativas = {"parecía", "tal que", "como"}
#separamos en oraciones y almacenamos las que contengan una de las palabras explicativas
oraciones_explicativas = []
for oracion in doc.sents:
    for token in oracion:
        if token.text.lower() in palabras_explicativas:
            oraciones_explicativas.append(oracion.text)
            break
#imprimimos las oraciones explicativas
oraciones_explicativas[0:5]

['El niño era otra buena razón para mantener alejados a los Potter: no querían que Dudley se juntara con un niño como aquél.\n',
 'Se volvió hacia los que murmuraban, como si quisiera decirles algo, pero se contuvo.\n',
 'Dejó el aparato y se atusó los bigotes mientras pensaba... No, se estaba comportando como un estúpido.',
 'No parecía disgustado por el empujón.',
 'Al contrario, su rostro se iluminó con una amplia sonrisa, mientras decía con una voz tan chillona que llamaba la atención de los que pasaban:\n\x97¡No se disculpe, mi querido señor, porque hoy nada puede molestarme! ¡Hay que alegrarse, porque Quien-usted-sabe finalmente se ha ido! ¡Hasta los muggles como usted deberían celebrar este feliz día!\nY el anciano abrazó al señor Dursley y se alejó.\n']

## Ejercicio 8

Con el modelo que estamos utilizando, los objetos de tipo `Token` a través de su atributo `morph` presentan información morfológica. En concreto, los verbos muestran (con más o menos precisión) entre otras cosas información sobre su tiempo verbal. Teniendo esto en cuenta cuántos verbos hay en cada tiempo verbal usando `token.tag_`.

In [23]:
mini_texto = "Ayer comí pizza. Hoy como con mi hermana, vamos a comer pescado."
mini_doc = nlp(mini_texto)
for token in mini_doc:
    print(f"{token} ---> {token.morph if token.morph else 'No tiene morfología'}")
    # Para sacar un atributo concreto de un objeto de tipo MorphAnalysis haremos ______.get("Tense")
    #almacena en un contador los token.morph
    contador[token.morph]+=1
print(contador)

Ayer ---> No tiene morfología
comí ---> Mood=Ind|Number=Sing|Person=3|Tense=Past|VerbForm=Fin
pizza ---> No tiene morfología
. ---> PunctType=Peri
Hoy ---> No tiene morfología
como ---> No tiene morfología
con ---> No tiene morfología
mi ---> Number=Sing|Number[psor]=Sing|Person=1|Poss=Yes|PronType=Prs
hermana ---> Gender=Fem|Number=Sing
, ---> PunctType=Comm
vamos ---> Mood=Ind|Number=Plur|Person=1|Tense=Pres|VerbForm=Fin
a ---> No tiene morfología
comer ---> VerbForm=Inf
pescado ---> Gender=Masc|Number=Sing
. ---> PunctType=Peri
Counter({'sentir': 75, 'reír': 18, 'llorar': 16, 'temer': 8, : 6, 'odiar': 5, 'sufrir': 4, 'amar': 2, PunctType=Peri: 2, Mood=Ind|Number=Sing|Person=3|Tense=Past|VerbForm=Fin: 1, Number=Sing|Number[psor]=Sing|Person=1|Poss=Yes|PronType=Prs: 1, Gender=Fem|Number=Sing: 1, PunctType=Comm: 1, Mood=Ind|Number=Plur|Person=1|Tense=Pres|VerbForm=Fin: 1, VerbForm=Inf: 1, Gender=Masc|Number=Sing: 1})


In [24]:
contador = Counter()


for token in doc:
    # Obtenemos la lista de tiempos verbales del token
    tiempos = token.morph.get("Tense")
    
    for tiempo in tiempos:
        contador[tiempo] += 1

print(contador)

Counter({'Past': 5169, 'Imp': 3802, 'Pres': 3285, 'Fut': 352})


## Ejercicio 9

Utiliza un `PhraseMatcher` para detectar menciones específicas dentro de un fragmento de texto basado en Harry Potter y la piedra filosofal. Las frases que se quieren detectar son expresiones mágicas conocidas como:
- piedra filosofal
- capa invisible

Usa `attr="LOWER"` en el `PhraseMatcher` para que las coincidencias no sean sensibles a mayúsculas

In [25]:
#crea un PhraseMatcher para detectar las menciones a 'piedra filosofal' y 'capa invisible'
from spacy.matcher import PhraseMatcher
matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
patron1 = nlp("piedra filosofal")
patron2 = nlp("capa invisible")
matcher.add("Piedra", [patron1])
matcher.add("Capa", [patron2])
#aplica el matcher al documento
matches = matcher(doc)
#imprime las oraciones que contienen 'piedra filosofal' y 'capa invisible'
for match_id, start, end in matches:
    span = doc[start:end]
    #imprime la oración que contiene el patrón
    print(span.sent.text)

Es una capa invisible dijo Ron, con una expresión de temor reverencial. Estoy seguro...
En cuanto se metió en la cama, pudo pensar libremente en ello: la capa invisible y quién se la había enviado.

13

Nicolás Flamel


Dumbledore había convencido a Harry de que no buscara otra vez el espejo de Oesed, y durante el resto de las vacaciones de Navidad la capa invisible permaneció doblada en el fondo de su baúl.
Nicolás Flamel susurró con tono teatral es el único descubridor conocido de la Piedra Filosofal.

Empujó el libro hacia ellos, y Harry y Ron leyeron:

El antiguo estudio de la alquimia está relacionado con el descubrimiento de la Piedra Filosofal, una sustancia legendaria que tiene poderes asombrosos.
Se ha hablado mucho de la Piedra Filosofal a través de los siglos, pero la única Piedra que existe actualmente pertenece al señor Nicolás Flamel, el notable alquimista y amante de la ópera.
? dijo Hermione, cuando Harry y Ron terminaron. El perro debe de estar custodiando la P

# Ejercicio 10

Utiliza expresiones regulares para localizar los encabezados de los capítulos. Cada capítulo está definido por un número (`1`, `2`, etc.) y un título. Además, te serán últiles las separaciones de las líneas y los espacios en blanco a la hora de definir la expresión regular. Con esta expresión definida, crear un objeto `Span` para cada capítulo utilizando `doc.char_span(...)`.

Definir tres atributos personalizados sobre cada `Span` de capítulo:
    - `.numero`: número del capítulo (como entero).
    - `.titulo`: título del capítulo (como cadena).
    - `.longitud`: longitud del capítulo en número de tokens.

In [52]:
import re
import pandas as pd
from spacy.tokens import Span
if not Span.has_extension("numero"):
    # Registrar atributos personalizados en Span
    Span.set_extension("numero", default=None)
    '''Span.set_extension("titulo", default=None)
    Span.set_extension("longitud", default=None)
# Registrar atributos personalizados en Span
Span.set_extension("numero", default=None)'''
if not Span.has_extension("titulo"):
    # Registrar atributos personalizados en Span
    Span.set_extension("titulo", default=None)
if not Span.has_extension("longitud"):
    # Registrar atributos personalizados en Span
    Span.set_extension("longitud", default=None)
#Buscar encabezados de capítulo con regex: número al principio de línea seguido de texto
# Ejemplo: "1 El niño que vivió"
capitulos = list(re.finditer(r"(?m)^(\d+)\s+([A-Z][^\n]*)", texto))

# Lista para guardar la info de cada capítulo
capitulos_info = []

# Iterar por capítulos para construir spans
for i, match in enumerate(capitulos):
    numero = int(match.group(1))
    titulo = match.group(2).strip()

    # Índices del capítulo actual en caracteres
    start_char = match.end()

    # Determinar el final del capítulo: inicio del siguiente o final del texto
    if i + 1 < len(capitulos):
        end_char = capitulos[i + 1].start()
    else:
        end_char = len(texto)

    # Crear el span usando char_span
    span = doc.char_span(start_char, end_char, alignment_mode="expand")

    if span:
        # Atributos personalizados
        span._.numero = numero
        span._.titulo = titulo
        span._.longitud = len(span)

        capitulos_info.append({
            "numero": numero,
            "titulo": titulo,
            "longitud": len(span),
        })

# Crear DataFrame
df = pd.DataFrame(capitulos_info)

# Mostrar resultado
print(df)

    numero                                        titulo  longitud
0        1                             El niño que vivió      5603
1        2                   El vidrio que se desvaneció      4165
2        3                           Las cartas de nadie      4596
3        4                     El guardián de las llaves      4493
4        5                            El callejón Diagon      8139
5        6  El viaje desde el andén nueve y tres cuartos      7708
6        7                     El sombrero seleccionador      5585
7        8                       El profesor de pociones      3666
8        9                         El duelo a medianoche      5929
9       10                                     Halloween      4995
10      11                                     Quidditch      4036
11      12                            El espejo de Oesed      6510
12      13                                Nicolás Flamel      3875
13      14                Norberto, el ridgeback noruego      